In [1]:
import os
import smtplib
from email.message import EmailMessage
from datetime import datetime
from getpass import getpass

import mysql.connector

conn = mysql.connector.connect(

host = 'localhost',
user = 'root',
passwd = os.environ.get("DB_PASS"),
database = 'bank')


def register():
    
    """
    This function for new users who wants to register on bank portal.
    """
    Username = input("Create Username : ")
    Password = getpass("Create Password : ")
    Password1 = getpass("Conform Password : ")
    
    if Password != Password1:
        print("Password don't match, Please Re-enter..")
        register()
        
    else:
        if len(Password) <= 6:
            print("Password too short, Please Re-enter.")
            register()
            
        else:
            mycursor = conn.cursor()
            mycursor.execute("SELECT User_name FROM authentic")
            lst = mycursor.fetchall()
            if (Username,) in lst:
                print('exists')
                register()
            else:
                mycursor = conn.cursor()
                query1 = "INSERT INTO authentic(User_name,Password) VALUES (%s, %s)"
                val1 = (Username, Password)
                
                mycursor.execute(query1, val1)
                conn.commit()
                print("Register successfully!")
                Main()


def access():
    """
    This function is used for Customer login.
      - If customer login successfully then he or she can perform all bank operations.
      - NOTE:- Before login it's mandatory to register.
    """
    Username = input("Enter Username : ")
    Password = getpass("Enter Password : ")
        
    mycursor = conn.cursor()
    mycursor.execute("SELECT User_name,Password FROM authentic")
    lst = mycursor.fetchall()
    if (Username,Password) in lst:
        print("Login Successful!")
        print("############### WELCOME TO SBI BANK ##################")
        main()
    else:
        print("Not Registered.Please register")
        register()
        

def Main():
    while True:
        print("1) Register \t\t 2) Login \t\t 3) Back")
        responce = int(input("Enter your responce: "))
        if responce == 1:
            register()
        elif responce == 2:
            access()
        elif responce == 3:
            break
        
               
def generate_account_no():
    
    """
    Auto generated account number using datetime module.
    return account number.
    """
    account_number = datetime.now().strftime("SBI350%Y%m%d%M%S")
    return account_number

def send_mail(email,n,ad,ob,account_number):
    """
    This Function send mail to user automatically when user create new account.
    """
    
    connection = smtplib.SMTP("smtp.gmail.com", 587)
    connection.ehlo()
    connection.starttls()
    connection.login((os.environ.get("mailuser")),(os.environ.get("mailpass")))
    message = EmailMessage()
    message['Subject'] = 'JD-SBIINB'
    message['From'] = os.environ.get('mailuser')
    message['To'] = email
    message.set_content(f"""
    <html>
    <body>
     Hii, <b>{n},</b> <br> &nbsp;&nbsp;&nbsp;&nbsp; Thank's for creating account in SBI.<br>
     Your account details:- <br>
     &nbsp;&nbsp;Account Number:- <b>{account_number},</b><br>
     &nbsp;&nbsp;Branch:- {ad}<br>
     &nbsp;&nbsp;Amount:- {ob}<br>
     <br>For more details please visit <a href="https://www.sbionline.com">www.sbionline.com</a>
    </body>
    """,subtype = 'html')
    connection.send_message(message)
    connection.quit()
    
            

def open_account():
    
    """
    This Function opens new account for customers.
    Return account number.
    
    """
   
    n = input("Enter name :- ")
    db = input("Enter D.O.B :- ")
    ad = input("Enter address :- ")
    ob = int(input("Enter opening balance :- "))
    pn = input("Enter phone number :- ")
    email = input("Enter you email:- ")
    
    mycursor = conn.cursor()
    mycursor.execute("SELECT Name,Phone_number FROM account")
    lst = mycursor.fetchall()
    
    if (n,pn) in lst:
        print("############ You are already member of SBI Bank. ##############")
        
    else:
    
        account_number = generate_account_no()
        date = datetime.now()

        mycursor = conn.cursor()

        query = "INSERT INTO account(Name,Account_number,Date_of_Birth,Phone_number,Opening_balance,Date,Address) VALUES(%s,%s,%s,%s,%s,%s,%s)"

        values = (n,account_number,db,pn,ob,date,ad)

        mycursor.execute(query,values)

        conn.commit()

        print(mycursor.rowcount, "record inserted")


        send_mail(email,n,ad,ob,account_number)


        print(f"Data entered Successfully.\n YOUR ACCOUNT NUMBER IS:-{account_number} ")

    
def deposite_amount():
    
    """
    This function deposite amount to customers account.
    Return available balance
    """
    
    am = int(input("Enter amount :- "))
    ac = input("Enter account number :- ")
    
    query2 = "SELECT Balance FROM amount WHERE Account_number = %s"
    val2 = (ac,)
    mycursor = conn.cursor()
    mycursor.execute(query2, val2)
    
    myresult = mycursor.fetchone()
    
    total_amount = myresult[0] + am
    
    query3 = "UPDATE amount SET Balance = %s WHERE Account_number = %s"
    
    val3 = (total_amount, ac)
    
    mycursor.execute(query3, val3)
    
    conn.commit()
    
    print(f"Amount Deposite Successfully.\n Available balance:-{total_amount}")
    
    
def withdraw_amount():
    
    """
    This Function withdraw amount from customers account.
    Return available balance.
    """
    am = int(input("Enter amount :- "))
    ac = input("Enter account number :- ")
    
    query4 = "SELECT Balance FROM amount WHERE Account_number = %s"
    val4 = (ac,)
    
    mycursor = conn.cursor()
    mycursor.execute(query4, val4)
    
    myresult = mycursor.fetchone()
    total_amount = myresult[0] - am
    
    query5 = "UPDATE amount SET Balance = %s WHERE Account_number = %s"
    val5 = (total_amount, ac)
    
    mycursor.execute(query5, val5)
    
    conn.commit()
    
    print(f"Amount withdraw Successfully.\n Available balance:-{total_amount}")
    
        
def balance():
    
    """
    This function shows balance from customers account.
    Return available balance.
    """
    ac = input("Enter account number :- ")
    query6 = "SELECT Balance FROM amount WHERE Account_number = %s"
    val6 = (ac,)
    
    mycursor = conn.cursor()
    
    mycursor.execute(query6, val6)
    myresult = mycursor.fetchone()
    
    print(f"Balance for account: {ac} is {myresult[0]}")
    
    
def display_account_details():
    
    """
    This function display all details of customers.
    Return list of data
    """
    
    ac = input("Enter account number :- ")
    query7 = "SELECT * FROM account WHERE Account_number = %s"
    val7 = (ac,)
    
    mycursor = conn.cursor()
    
    mycursor.execute(query7, val7)
    
    print(f"Your Account Details is \n{mycursor.fetchall()}")
       
    
def close_account():
    
    """
    This function is used when customer want to delete account.
    """
    
    ac = input("Enter account number :- ")
    query8 = "DELETE FROM account WHERE Account_number = %s"
    query9 = "DELETE FROM amount WHERE Account_number = %s"
    val9 = (ac,)
    
    mycursor = conn.cursor()
    
    mycursor.execute(query8, val)
    mycursor.execute(query9, val)
    
    conn.commit()
    
    print("Account closed successfully.")
    
    
def main(): 
    
    """
    This is main function.
    User can choose verious options given in this function.
    """
    
    while True:
    
        print("""
        1. OPEN NEW ACCOUNT
        2. DEPOSITE AMOUNT
        3. WITHDRAW AMOUNT
        4. BALANCE ENQUIRY
        5. DISPLAY ACCOUNT DETAILS
        6. CLOSE ACCOUNT
        7. EXIT
        
        """)
        choice = int(input("############### WELCOME TO SBI ################## \n Enter Task Number :- "))
        if choice == 1:
            open_account()
         
        elif choice == 2:
            deposite_amount()
        
        elif choice == 3:
            withdraw_amount()
        
        elif choice == 4:
            balance()
    
        elif choice == 5:
            display_account_details()
        
        elif choice == 6:
            close_account()
           
        elif choice == 7:
            print("############# THANK'S FOR VISITING SBI BANK. #################")
            break
        
        else:
            print("Wrong choice\n Please Enter from below choices: ")
            
Main()

1) Register 		 2) Login 		 3) Back
Enter your responce: 2
Enter Username : Shri Ganesh
Enter Password : ········
Login Successful!
############### WELCOME TO SBI BANK ##################

        1. OPEN NEW ACCOUNT
        2. DEPOSITE AMOUNT
        3. WITHDRAW AMOUNT
        4. BALANCE ENQUIRY
        5. DISPLAY ACCOUNT DETAILS
        6. CLOSE ACCOUNT
        7. EXIT
        
        
############### WELCOME TO SBI ################## 
 Enter Task Number :- 3
Enter amount :- 100
Enter account number :- SBI350202110072235


TypeError: 'NoneType' object is not subscriptable

In [6]:
import mysql.connector
import os

conn = mysql.connector.connect(

host = 'localhost',
user = 'root',
passwd = os.environ.get("DB_PASS"),
database = 'bank')




ac = input("Enter account_number: ")

query4 = "SELECT Balance FROM amount WHERE Account_number = ac"
# val4 = (ac,)

mycursor = conn.cursor()
mycursor.executemany(query4)

# conn.commit()



Enter account_number: SBI350202110072235


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '%s' at line 1